In [1]:
#Importing necessary Libraries
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import PIL

import tensorflow as tf
from tensorflow.keras.layers import *
import tensorflow_addons as tfa
from keras.models import Sequential

In [2]:
TRAIN_PATH = "../input/pneumonia-classification-challenge/pneumonia_dataset/pneumonia_dataset/train"
#TRAIN_PATH = "../input/pneumonia-classification-challenge/pneumonia_dataset_new/pneumonia_dataset_new"
TEST_PATH = "../input/pneumonia-classification-challenge/pneumonia_dataset/pneumonia_dataset/test"

BATCH_SIZE = 32
HEIGHT,WIDTH = 200,200
CHANNELS = 3
NUM_CLASSES = 2
SEED = 143

In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range = 0.2, 
    zoom_range = 0.2,
    validation_split=0.2
   )

train_ds = train_datagen.flow_from_directory(
                             TRAIN_PATH,
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "binary",
                             shuffle = True,
                             seed = SEED,
                             subset = "training",
                             color_mode='rgb'
                            )
val_ds = train_datagen.flow_from_directory(
                             TRAIN_PATH,
                             target_size = (HEIGHT,WIDTH),
                             batch_size = BATCH_SIZE,
                             class_mode = "binary",
                             shuffle = True,
                             seed = SEED,
                             subset = "validation",
                             color_mode='rgb'
                            )

test_df = pd.read_csv("../input/pneumonia-classification-challenge/pneumonia_dataset/pneumonia_dataset/test.csv")
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
   )
test_ds = test_datagen.flow_from_dataframe(
                      test_df,
                      directory = TEST_PATH,
                      x_col = "filename",
                      y_col = "filename",
                      target_size = (HEIGHT,WIDTH),
                      shuffle = False,
                      batch_size = BATCH_SIZE
                       )


classes_dict = train_ds.class_indices
print(classes_dict)

Found 1940 images belonging to 2 classes.
Found 485 images belonging to 2 classes.
Found 606 validated image filenames belonging to 606 classes.
{'normal': 0, 'pneumonia': 1}


In [4]:
model = tf.keras.applications.DenseNet121(weights= "imagenet",
                                    include_top=False,
                                    input_shape=(HEIGHT,WIDTH,CHANNELS), pooling="avg")
predictions = tf.keras.layers.Dense(14, activation='sigmoid', name='predictions')(model.output)
model = tf.keras.Model(inputs=model.input, outputs=predictions)   
model.load_weights("../input/pneumonia-classification-challenge/pretrained.h5")
model = tf.keras.Model(model.input, model.layers[-2].output) 
x = tf.keras.layers.Dense(512, activation = "relu")(model.output)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(128, activation = "relu")(x)
x = tf.keras.layers.Dense(64, activation = "relu")(x)
outputs = tf.keras.layers.Dense(1, activation = "sigmoid", dtype = tf.float32)(x)
chxnet = tf.keras.Model(model.input,outputs)

for layer in chxnet.layers[:-15]:
        layer.trainble = False

29089792/29084464 [==============================] - 1s 0us/step


In [5]:
def create_model():
    inputs = tf.keras.Input(shape = (HEIGHT,WIDTH,3))
    x = Conv2D(32,(3,3),activation = "relu") (inputs)
    x = MaxPooling2D((2,2)) (x)
    x = Conv2D(64,(3,3),activation = "relu") (x)
    x = MaxPooling2D((2,2)) (x)
    x = Conv2D(128,(3,3),activation = "relu") (x)
    x = MaxPooling2D((2,2)) (x)
    x = Conv2D(128,(3,3),activation = "relu") (x)
    x = MaxPooling2D((2,2)) (x)
    x = Flatten() (x)
    x = Dense(512,activation = "relu") (x)
    outputs = Dense(1,activation = "sigmoid") (x)
    
    model = tf.keras.Model(inputs,outputs)
    return model

model = create_model()
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 198, 198, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 128)       0   

In [6]:
def compile_model(model, lr=0.001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = "binary_crossentropy"
    
    metrics = [
       'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [7]:
METRIC = "val_accuracy"

def create_callbacks():
    
    cpk_path = f'./best_model.h5'
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor= METRIC,
        mode='max',
        save_best_only=True,
        verbose=1,
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= METRIC,
        mode='max',
        factor=0.1,
        patience=1,
        verbose=1
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= METRIC,
        mode='max',
        patience=10, 
        verbose=1
    )
    
    
    callbacks = [checkpoint, reducelr , earlystop]         
    
    return callbacks

In [8]:
EPOCHS= 50
VERBOSE =1

tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    
    model = create_model()
    model = compile_model(model,lr=0.001)
   
    callbacks = create_callbacks()
    
    history = model.fit(train_ds, 
                        epochs=EPOCHS,
                        callbacks=callbacks,
                        validation_data = val_ds,
                        verbose=VERBOSE
                       )

Epoch 1/50
61/61 [==============================] - 99s 2s/step - loss: 0.7540 - accuracy: 0.5240 - precision: 0.4701 - recall: 0.2046 - val_loss: 0.6915 - val_accuracy: 0.5278 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00

Epoch 00001: val_accuracy improved from -inf to 0.52784, saving model to ./best_model.h5
Epoch 2/50
61/61 [==============================] - 64s 1s/step - loss: 0.6928 - accuracy: 0.5235 - precision: 0.3587 - recall: 0.0260 - val_loss: 0.6902 - val_accuracy: 0.5278 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00

Epoch 00002: val_accuracy did not improve from 0.52784

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 3/50
61/61 [==============================] - 61s 1s/step - loss: 0.6884 - accuracy: 0.5308 - precision: 0.2472 - recall: 0.0031 - val_loss: 0.6868 - val_accuracy: 0.5485 - val_precision: 0.7083 - val_recall: 0.0742

Epoch 00003: val_accuracy improved from 0.52784 to 0.54845, saving model to ./best_model.

In [9]:
MODEL_PATH = "./best_model.h5"

model = tf.keras.models.load_model(MODEL_PATH)
classes_dict = {0 : 'normal', 1 : 'pneumonia'}

pred = model.predict(test_ds)
pred_ls = ['normal' if i < 0.5 else 'pneumonia' for i in pred]

test_df["label"] = pred_ls
test_df.to_csv("submission.csv",index = False)
test_df.head()

,filename,label
0,CXR_test_519.png,pneumonia
1,CXR_test_578.png,normal
2,CXR_test_359.png,normal
3,CXR_test_573.png,pneumonia
4,CXR_test_471.png,normal
